# 사전학습된 임베딩 계층을 이용한 자연어 처리

이 노트북은 사전 학습된 Word2Vec (GoogleNews-vectors-negative300.bin) 임베딩을 사용하여 텍스트 분류 모델을 구축하는 과정을 보여줍니다. IMDB 영화 리뷰 데이터셋을 사용하여 긍정/부정 감성 분석을 수행합니다.

## 1. 라이브러리 설치 및 데이터 준비

### 1.1. 필요한 라이브러리 설치

gensim 라이브러리가 필요합니다. 다음 명령어를 사용하여 설치할 수 있습니다:
```bash
conda install gensim
```

### 1.2. Word2Vec 모델 다운로드

GoogleNews-vectors-negative300.bin.gz 파일을 다운로드해야 합니다. 다음 링크 중 하나를 사용할 수 있습니다:

- **Direct Link (S3 AWS):** https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
- **Hugging Face:** NathaNn1111/word2vec-google-news-negative-300-bin와 같은 저장소에서 다운로드 링크를 제공합니다.
- **Kaggle:** https://www.kaggle.com/datasets/leadbest/googlenewsvectorsnegative300

다운로드한 파일은 `./data/` 디렉토리에 `GoogleNews-vectors-negative300.bin`으로 저장해야 합니다.

### 1.3. IMDB 데이터셋 다운로드 및 전처리

IMDB 영화 리뷰 데이터셋을 다운로드하고 압축을 해제한 후, 훈련 및 검증 세트로 분할합니다.

In [ ]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from keras.layers import TextVectorization
import os, pathlib, shutil, random
import keras
from gensim.models import KeyedVectors
import numpy as np

# 데이터 다운로드
def download():
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"

    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)

    print("Download complete!")

# 압축풀기
def release():
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"])
    print("압축풀기 완료")

# 라벨링 (train/val 분리)
def labeling():
    base_dir = pathlib.Path("aclImdb")
    val_dir = base_dir/"val"
    train_dir = base_dir/"train"

    for category in ("neg", "pos"):
        os.makedirs(val_dir/category, exist_ok=True)
        files = os.listdir(train_dir/category)
        random.Random(1337).shuffle(files)
        num_val_samples = int(0.2 * len(files))
        val_files = files[-num_val_samples:]
        for fname in val_files:
            shutil.move(train_dir/category/fname, val_dir/category/fname)

# 데이터 다운로드 및 전처리 실행 (필요시 주석 해제)
# download()
# release()
# labeling()

print("데이터 다운로드 및 전처리 함수 정의 완료. 필요시 주석을 해제하여 실행하세요.")

## 2. 데이터셋 로드 및 전처리

IMDB 데이터셋을 Keras `text_dataset_from_directory`를 사용하여 로드하고, `TextVectorization` 레이어를 사용하여 텍스트를 정수 시퀀스로 변환합니다.

In [ ]:
# 데이터셋 로드
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train",
    batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val",
    batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test",
    batch_size=batch_size
)

print("
데이터셋 샘플 확인:")
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[:3])
    print("targets[0]:", targets[:3])
    break

# 텍스트 벡터화
max_length = 600
max_tokens = 20000

text_vectorization = TextVectorization(
    max_tokens = max_tokens,
    output_mode = "int",
    output_sequence_length = max_length
)

text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds) # 어휘 사전 구축

int_train_ds = train_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE )
int_val_ds = val_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE )
int_test_ds = test_ds.map( lambda x,y:(text_vectorization(x), y), num_parallel_calls=tf.data.AUTOTUNE )

print("
정수 인코딩된 데이터셋 샘플 확인:")
for item in int_train_ds:
    print(item)
    break

## 3. Word2Vec 임베딩 로드 및 임베딩 행렬 생성

사전 학습된 Word2Vec 모델을 로드하고, 이를 기반으로 Keras `Embedding` 레이어에 사용할 임베딩 행렬을 생성합니다.

In [ ]:
# Word2Vec 파일 불러오기
filename = "./data/GoogleNews-vectors-negative300.bin"

try:
    word2vec_model = KeyedVectors.load_word2vec_format(filename, binary=True)
    embedding_dim = word2vec_model.vector_size
    print("Word2Vec 파일 로딩 성공. 임베딩 차원:", embedding_dim)
except FileNotFoundError:
    print(f"오류: {filename}을(를) 찾을 수 없습니다. 파일을 `./data/` 디렉토리에 올바르게 배치했는지 확인하세요.")
    exit()
except Exception as e :
    print(f"에러 발생: {e}")
    exit()

# 임베딩 행렬 생성
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))

for word, i in word_index.items():
    if i < max_tokens:
        try:
            embedding_vector = word2vec_model[word]
            embedding_matrix[i] = embedding_vector
        except KeyError:
            pass  # Word2Vec 모델에 없는 단어는 건너뜀

print("
생성된 임베딩 행렬의 일부 (상위 10개 단어):")
print(embedding_matrix[:10])

## 4. 모델 구축 및 훈련

사전 학습된 임베딩 레이어를 사용하여 양방향 LSTM 모델을 구축하고 훈련합니다.

In [ ]:
# 모델 구축
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim = max_tokens,
    output_dim=embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,  # 사전 학습된 임베딩은 훈련 중 업데이트하지 않음
    mask_zero=True
)(
    inputs
)

print("임베딩 레이어 출력 형태:", embedded.shape)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)

# 모델 컴파일
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

print("
모델 요약:")
model.summary()

# 모델 훈련
print("
모델 훈련 시작...")
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=15)

print("
테스트셋 평가:")
print("테스트셋 결과:", model.evaluate(int_test_ds))